****Questo notebook è stato pensato per essere eseguito su Colab.****

In [ ]:
import json
import nltk
import numpy as np
import random

from google.colab import files
from nltk import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!pip install sentence-transformers
# !pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

### Upload dei dati

In [18]:
corpus = list()

files.upload() 
files.upload()
files.upload()

carica_file = list()
carica_lore = list()
carica_ludo = list()

# articoli iss
with open("file_Stefano.txt", 'r', encoding='utf-8') as f:
  for line in f:
    carica_file.append(line)

# articoli / news ministero della salue
with open("file_Lorenzo.txt", 'r', encoding='utf-8') as lo:
  articoliSeparati = json.loads(lo.read())
  for articolo in articoliSeparati:
    for frase in articolo:
        carica_lore.append(frase)

# FAQ
with open('q&a.json', 'r') as ludo:
    q_a = json.load(ludo)
    for k in q_a.keys():
      carica_ludo.append(k)
  
corpus = carica_file + carica_lore + carica_ludo

Saving file_Stefano.txt to file_Stefano (1).txt


Saving file_Lorenzo.txt to file_Lorenzo (1).txt


Saving q&a.json to q&a (1).json


### Preprocessing
**corpus** è la lista dei documenti/articoli che sono stati raccolti.

**doc_corpus** è una lista contenente liste. Ogni lista contiene le frasi di un documento/articolo

**sentence_corpus** è la lista contenente tutte le frasi del corpus.

In [19]:
# SENTENCE SPLITTING
doc_corpus = list()
sentence_corpus = list()

for line in corpus:
  sentence_doc_list = nltk.sent_tokenize(line)
  doc_corpus.append(sentence_doc_list)

for doc in doc_corpus:
  for sentence in doc:
    sentence_corpus.append(sentence.lower())

In [ ]:
# ogni line abbiamo un documento
for l in corpus[:10]:
  print(l)

La pandemia da COVID-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita. Per questo motivo, l’ISS mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento. In evidenza:  la pagina dedicata alla vaccinazione contro il COVID-19 in gravidanza e allattamentola pagina dedicata allo studio “COVID-19 e salute mentale perinatale: impatto del COVID-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i Consultori Familiari (CF)”, di cui l’Istituto Superiore di Sanità è coordinatore, aggiornata al 7 gennaio 2021Leggi gli altri approfondimenti:Per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’ISS hanno messo a punto alcuni materiali di comunicazione specifici. Visita lapagina dedicata.Data di ultimo aggiornamento: 9 gennaio 2021 Data di creazio

In [ ]:
# ogni line abbiamo una lista di frasi
for l in doc_corpus[:10]:
  print(l)

['La pandemia da COVID-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita.', 'Per questo motivo, l’ISS mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento.', 'In evidenza:  la pagina dedicata alla vaccinazione contro il COVID-19 in gravidanza e allattamentola pagina dedicata allo studio “COVID-19 e salute mentale perinatale: impatto del COVID-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i Consultori Familiari (CF)”, di cui l’Istituto Superiore di Sanità è coordinatore, aggiornata al 7 gennaio 2021Leggi gli altri approfondimenti:Per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’ISS hanno messo a punto alcuni materiali di comunicazione specifici.', 'Visita lapagina dedicata.Data di ultimo aggiornamento: 9 gennaio 2021 Data

In [ ]:
sentence_corpus[:10]

['la pandemia da covid-19 ha reso necessaria una riorganizzazione dei percorsi assistenziali legati alla nascita.',
 'per questo motivo, l’iss mette a disposizione del personale sanitario una selezione di risorse, letteratura ed evidence disponibili su gravidanza, parto e allattamento.',
 'in evidenza:  la pagina dedicata alla vaccinazione contro il covid-19 in gravidanza e allattamentola pagina dedicata allo studio “covid-19 e salute mentale perinatale: impatto del covid-19 sul vissuto e lo stato emotivo in epoca perinatale delle donne in contatto con i consultori familiari (cf)”, di cui l’istituto superiore di sanità è coordinatore, aggiornata al 7 gennaio 2021leggi gli altri approfondimenti:per aiutare il personale sanitario nel lavoro quotidiano di assistenza alle donne durante la gravidanza, il parto e l’allattamento, le ricercatrici dell’iss hanno messo a punto alcuni materiali di comunicazione specifici.',
 'visita lapagina dedicata.data di ultimo aggiornamento: 9 gennaio 2021 d

### Costruzione del modello - BERT XXL



Il modello è stato preso da https://huggingface.co/dbmdz/bert-base-italian-cased

In [21]:
bert_model = SentenceTransformer('dbmdz/bert-base-italian-xxl-uncased') 

Exception when trying to download https://sbert.net/models/dbmdz/bert-base-italian-xxl-uncased.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/dbmdz/bert-base-italian-xxl-uncased.zip not found. Try to create it from scratch
Try to create Transformer Model dbmdz/bert-base-italian-xxl-uncased with mean pooling


A differenza di word2vec questo modello non richiede che il testo sia processato (gli passiamo la lista delle frasi del nostro corpus di documenti)

In [22]:
sentence_embeddings = bert_model.encode(sentence_corpus, 
                                        show_progress_bar=True)

In [23]:
# funzione che calcola la cosine similarity
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

### Costruzione del modello RoBERTa
Il modello è stato preso da https://huggingface.co/sentence-transformers/msmarco-distilroberta-base-v2

In [ ]:
bert_model2 = SentenceTransformer('msmarco-distilroberta-base-v2')

100%|██████████| 305M/305M [00:14<00:00, 20.8MB/s]


In [ ]:
sentence_embeddings2 = bert_model2.encode(sentence_corpus, 
                                          show_progress_bar=True)

### Impostazione del chatbot

In [24]:
# funzione che restituisce un saluto come risposta a un saluto dell'utente
def greeting_response(text):
    text = text.lower()
    
    # lista dei possibili saluti che il bot può dare
    bot_greetings = ['ciao', 'ehi']
    # lista dei saluti che il bot riconosce
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [25]:
def bot_response(user_input, model,embeddings, q_a, corpus):
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    found_faq = False
    
    # vettorizziamo l'input
    input_emb = model.encode([user_input])[0]
    # answer_list = lista che conterrà tutte le frasi che con similarità > 0.35
    answers_list = list()
    
    for i in range(len(embeddings)):
      sent = corpus[i]
      # calcoliamo la similarità tra la query e ogni
      # frase (embedding) del corpus
      sim = cosine(input_emb, embeddings[i])
      if sim > 0.35:
        answers_list.append(((sim, sent)))
      i += 1

    # riordiniamo la lista delle possibili risposte a seconda dello score di similarità 
    answers_list.sort(key=lambda x : x[0], reverse=True)
    
    # prima controlliamo se ci sono delle FAQ
    for k in q_a.keys():
      if answers_list[0][1] == k.lower():
        bot_response = bot_response + q_a[k][0]
        response_flag = 1
        found_faq = True
        return bot_response
   
    i = 0
    quest = True
    if not found_faq:
      while quest:
        if '?' in answers_list[i][1]:
          i += 1
        else:
          bot_response = answers_list[i][1]
          response_flag = 1
          quest = False
        
    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'
    
    return bot_response

### Esempio di conversazione (try it yourself!)

In [ ]:
####### PER CAMBIARE MODELLO ###################################
####### cambiare model e sentence_embedding ####################
####### bert_model e sentence_embeddings (x BERT XXL) ###########
####### bert_model2 e sentence_embeddings2 (x RoBERTa) #########

print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
while(True):
    print()
    print('User:')
    user_input = input()
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, bert_model, sentence_embeddings ,q_a, sentence_corpus))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:
cosa devo fare se ho i sintomi del coronavirus?
Co-bot: ... 
I coronavirus sono virus che circolano tra gli animali e alcuni di essi infettano anche l’uomo. I pipistrelli sono considerati ospiti naturali di questi virus, ma anche molte altre specie di animali sono considerate fonti. Ad esempio, il Coronavirus della sindrome respiratoria del Medio Orientale (MERS-CoV) viene trasmesso all’uomo dai cammelli e la sindrome respiratoria acuta grave Coronavirus-1 (SARS-CoV-1) viene trasmesso all’uomo dallo zibetto.

User:
quali vaccini esistono
Co-bot: ... 
Le tempistiche e le cifre potranno essere soggette a variazioni in funzione dei processi di autorizzazione e assegnazione delle dosi. L’Italia, in base agli accordi stipulati, potrà contare sulla disponibilità di 215 milioni di dosi. Il primo vaccino ad essere autorizzato in Unione Europea è stato Comirnaty di Pfizer-BioNtech il 21 dicembre. Le vaccinazioni 

### Testing
Il nostro test set è una lista di 42 domande che sono state raccolte con le modalità descritte nel report.

In [1]:
test_set = ["Cosa fare nel caso in cui si manifestano i sintomi?", 
            "Posso viaggiare se ho il coronavirus?", 
            "Le donne in gravidanza possono trasmettere la malattia al figlio?", 
            "Quanto tempo resta nel corpo il coronavirus?", 
            "Come funziona il contact tracing?", 
            "Quando finirà la pandemia?", 
            "Il vaccino ha effetti collaterali?", 
            "Cosa sappiamo della nuova variante inglese?", 
            "Quanti tipi di vaccino esistono?", 
            "Fino a quando saranno chiuse le attività commerciali?", 
            "Come distinguiamo un'influenza stagionale da un'influenza covid?", 
            "Chi si è vaccinato può trasmette la malattia?", 
            "Quali sono i colori che possono assumere le regioni?", 
            "Come fanno le persone infette a trasmettere il virus?", 
            "Se vengo dall'estero devo fare il tampone?", 
            "Cos’è l'indice Rt?", 
            "Posso ammalarmi due volte di covid?", 
            "Quanto tempo dura l'isolamento fiduciario?", 
            "È possibile fare sport di contatto?", 
            "Come faccio a capire se sono positiva ma asintomatica?", 
            "Sono stato a contatto con un infetto: cosa devo fare?", 
            "Fino a quando resteranno chiuse le piscine e le palestre?", 
            "Dopo il vaccino, devo comunque continuare a usare la mascherina?", 
            "Che cosa devo fare se voglio andare all'estero?", 
            "Lo smog/inquinamento influisce sulla diffusione del coronavirus/covid?", 
            "Qual è il periodo di copertura del vaccino?", 
            "Se sono all’estero, cosa devo fare per tornare in Italia?", 
            "Come proteggere le persone più anziane dal virus?", 
            "Il coronavirus è pericoloso per gli animali domestici?", 
            "La mascherina è obbligatoria all’aperto?", 
            "Fino a quando sarà adottata la didattica a distanza per le scuole superiori?", 
            "Qual è l'incidenza del covid nei bambini?", 
            "I guanti servono a proteggere dal contagio?", 
            "Si può viaggiare fuori comune in area gialla?", 
            "Quanto sono importanti il distanziamento e la chiusura delle attività nella trasmissione del covid?", 
            "Posso fare attività sportiva se sono positivo asintomatico?", 
            "Per i bambini è obbligatorio l’uso della mascherina?", 
            "Come agiscono i linfociti T nelle infezioni da covid?", 
            "Posso visitare i miei nonni?", 
            "È possibile andare a pesca?", 
            "Quali sono i sintomi del coronavirus?", 
            "Il virus può rimanere sulle superfici?"]

In [28]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

for sentence in test_set:
    user_input = sentence
    
    print()
    print('User:')
    print (user_input)

    print('Co-bot: ... ')
    print(bot_response(user_input, bert_model, 
                       sentence_embeddings,
                       q_a, sentence_corpus))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus



user 1
Cosa fare nel caso in cui si manifestano i sintomi?
Co-bot: ... 
La scuola dovrà pulire e disinfettare le superfici della stanza o area di isolamento dopo che l’alunno con i sintomi è tornato a casa. Una volta riportato il figlio/a a casa i genitori devono contattare il Pediatra di libera scelta (PLS) o il Medico di medicina generale (MMG) che, dopo avere valutato la situazione, deciderà se è necessario contattare il Dipartimento di prevenzione (DdP) per l’esecuzione del tampone.


user 2
Posso viaggiare se ho il coronavirus?
Co-bot: ... 
inoltre, per facilitare la comunicazione, il gruppo di studio sul coronavirus ha proposto di utilizzare, per i singoli isolati, la convenzione di classificazione: sars-cov-2/isolato/ospite/data/luogo.


user 3
Le donne in gravidanza possono trasmettere la malattia al figlio?
Co-bot: ... 
La gravidanza comporta cambiamenti del sistema immunitario che possono aumentare 